In [1]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <85021449-F141-385B-8151-410662B4D328> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
batch_size = 100
training_data = datasets.FashionMNIST("./fashion_mnist", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST("./fashion_mnist", train=False, download=True, transform=transforms.ToTensor())

train_dataloader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=2)

In [3]:
# hyperparameters
# 28 x 28
sequence_len = 28 # 입력(사이즈 28)이 28개의 시퀀스로 구성된 데이터
input_len = 28 
hidden_size = 128
num_layers = 2
num_classes = 10
num_epoches = 5
lr = 0.01

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_classes, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes) # 입력 : hidden_size, 출력 : num_classes

    def forward(self, x):
        hidden_states = torch.zeros(num_layers, x.size(0), self.hidden_size) # hidden_layer 개수, sample 개수, hidden_layer 크기
        cell_states = torch.zeros(num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (hidden_states, cell_states)) # input : x, 초기 상태 : (hidden_states, cell_states), out : 모든 time step(sequence)의 출력, _ : 은닉 상태 및 셀 상태
        out = self.output_layer(out[:,-1,:]) # 각 입력 시퀀스에 대해 마지막 time step의 출력
        return out

In [5]:
model = LSTM(input_len, hidden_size, num_classes, num_layers)

In [6]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr)
adam = optim.Adam(model.parameters(), lr)

In [7]:
def train(num_epoches, model, train_dataloader, loss_func):
    total_batches = len(train_dataloader) # batch의 개수

    for epoch in range(num_epoches):
        for batch, (image, labels) in enumerate(train_dataloader): # train_dateloader : val, label
            image = image.reshape(-1, sequence_len, input_len) # sample 개수, sequence!!, input
            output = model(image)

            print(output.shape)
            print(labels.shape)
            loss = loss_func(output, labels)

            # 이 순서를 지켜야 함.
            adam.zero_grad()
            loss.backward()
            adam.step()

            if (batch + 1) % 100 == 0:
                print(f"Epoch: {epoch}; Batch: {batch + 1} / {total_batches}; Loss: {loss.item():>4f}")

In [8]:
train(num_epoches, model, train_dataloader, loss) # num과 size는 다르다. 

/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <85021449-F141-385B-8151-410662B4D328> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/an

torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])


/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <85021449-F141-385B-8151-410662B4D328> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/an

torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])


/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <85021449-F141-385B-8151-410662B4D328> /opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/anaconda3/envs/pytorch_unet/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/an

torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])
torch.Size([100, 10])
torch.Size([100])


KeyboardInterrupt: 

In [ ]:
test_loss = 0
with torch.no_grad():
    for X, y in test_dataloader:
        X = X.reshape(-1, 28, 28)
        pred = model(X)
        test_loss += loss(pred, y).item()

print(test_loss)